Будем использовать физическими величинами с измерениями. Модуль PhysicalQuantities сломался, не обновлялся два года, поэтому придется попробовать `pint` + `uncertainties`.

In [1]:
from pint import UnitRegistry
from uncertainties import ufloat, umath
from scipy.constants import k as k_J_per_K
import sympy as sp

In [2]:
ureg = UnitRegistry()

In [3]:
H_отсчета = (ufloat(200, 1) * ureg.kilometer).to_base_units()
H_отсчета

<Quantity(200000+/-1000, 'meter')>

In [4]:
P_отсчета =  (ufloat(8, 1)* 1e-5 * ureg.pascal).to_base_units()
P_отсчета

<Quantity((8.00000+/-1.00000)e-05, 'kilogram / meter / second ** 2')>

In [5]:
H = (ufloat(70, 10) * ureg.kilometer).to_base_units()
H

<Quantity(70000.0+/-10000.0, 'meter')>

In [6]:
σ = (ufloat(8, 2) * ureg.angstrom * ureg.angstrom).to_base_units()
σ

<Quantity((8.00000+/-2.00000)e-20, 'meter ** 2')>

In [7]:
T_отсчета = (ufloat(854.4, 200) * ureg.K).to_base_units()
T_отсчета

<Quantity(854.400+/-200.000, 'kelvin')>

In [8]:
T_экзосферы = (ufloat(1000, 300) * ureg.kelvin).to_base_units()
T_экзосферы

<Quantity(1000.00+/-300.00, 'kelvin')>

Todo: разобраться, почему `ureg` не может сам разобраться с конверсией единиц измерения, и почему там константы типа Units, а не Quantities

In [9]:
k = (k_J_per_K * ureg.J / ureg.K).to_base_units()
k

<Quantity(1.380649e-23, 'kilogram * meter ** 2 / second ** 2 / kelvin')>

In [10]:
p_экз = k * T_экзосферы /  σ / H
p_экз.to('pascal')

<Quantity((2.46544+/-1.02519)e-06, 'pascal')>

In [11]:
h_экз = H_отсчета + H * umath.log( (P_отсчета / p_экз).magnitude )
h_экз.to('km')

<Quantity(443.576+/-53.212, 'kilometer')>

In [12]:
h_экз = h_экз.to_base_units()
h_экз

<Quantity(443576+/-53212, 'meter')>

----

In [13]:
G = (1 * ureg.gravitational_constant).to_base_units()
G

<Quantity(6.6743e-11, 'meter ** 3 / kilogram / second ** 2')>

In [14]:
M_земли = ufloat(5.9722e24, 1.2e21) * ureg.kg
M_земли

<Quantity((5.97220+/-0.00120)e+24, 'kilogram')>

In [15]:
R_земли = (ufloat(6.371e6, 1e4) * ureg.meter).to_base_units()
R_земли

<Quantity((6.37100+/-0.01000)e+06, 'meter')>

In [16]:
R_земли+h_экз

<Quantity((6.81458+/-0.05414)e+06, 'meter')>

In [17]:
v_esc = umath.sqrt(2*G*M_земли/(R_земли+h_экз))
v_esc

DimensionalityError: Cannot convert from 'meter ** 2 / second ** 2' to 'dimensionless'

----

In [ ]:
m, k, T = sp.symbols('m, k, T', positive=True)  # Mass, Boltzmann constant, temperature
v, theta, phi = sp.symbols('v, theta, phi', real=True)  # Spherical coordinates
m, k, T, v, theta, phi

(m, k, T, v, theta, phi)

In [ ]:
норм_коэф = (m / (2 * sp.pi * k * T))**(3/2)
норм_коэф


0.353553390593274*m**1.5/(pi**1.5*T**1.5*k**1.5)

In [ ]:
f = норм_коэф * sp.exp(-m * v**2 / (2 * k * T)) * v**2 * sp.sin(theta)
f

0.353553390593274*m**1.5*v**2*exp(-m*v**2/(2*T*k))*sin(theta)/(pi**1.5*T**1.5*k**1.5)

Интегрируем по $\phi$ от $-\pi$ до $\pi$

In [ ]:
интеграл_phi = sp.integrate(f, (phi, -sp.pi, sp.pi))
интеграл_phi

0.707106781186548*m**1.5*v**2*exp(-m*v**2/(2*T*k))*sin(theta)/(pi**0.5*T**1.5*k**1.5)

Интегрируем по $\theta$ от $0$ до $\pi$

In [ ]:
интеграл_theta = sp.integrate(int_phi, (theta, 0, sp.pi))
интеграл_theta

1.4142135623731*m**1.5*v**2*exp(-m*v**2/(2*T*k))/(pi**0.5*T**1.5*k**1.5)

Интегрируем по $v$ от $0$ до $\infty$

In [ ]:
рез = sp.integrate(int_theta, (v, 0, sp.oo))
рез


0.707106781186548*sqrt(2)

Интегрируем по $\theta$ от $0$ до $\pi$